对数据进行预处理和加载是一件非常重要的事情，在pytorch中，使用类**Dataset**和**DataLoader**，其所在的package是**torch.utils.data**。

## Dataset
Dataset是一个抽象类，在用户需要加载自定义数据的时候进行继承并重写如下两个方法：
- \_\_getitem\_\_: 用来获取数据集索引的数据，例如dataset[i]返回数据集第i个样本。
- \_\_len\_\_: 实现返回整个数据集的大小。

也就是说Dataset的\_\_getitem\_\_函数实现了数据的方式(例如有时数据读取需要不同格式文件的读取操作等)以及定义了每个每个item的内容(例如item可以包含一个image_tensor及label_tensor)。

## DataLoader
定义好Dataset后，就可以使用DataLoader去加载数据了。
其参数定义如下：
```
class torch.utils.data.DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    sampler=None,
    batch_sampler=None,
    num_workers=0,
    collate_fn=<function default_collate>,
    pin_memory=False,
    drop_last=False,
    timeout=0,
    worker_init_fn=None)
```
- num_workers:即加载数据时制定进程数量(=0表示自动设置)，在Windows下该参数设置为0，否则运行报错。

### collate_fn
该函数明确了数据集加载过程中进行的处理操作。

函数的参数是一个list，list中的每个原始就是Dataset里面定义的\_\_getitem\_\_这个函数的返回值。
```
def collate_fn(batch):
    '''
    对batch进行处理和操作
    '''
    return batch
```
通过collate_fn可以对一个batch进行进一步的处理

## Example:

### 示例1

In [5]:
import torch
from torch.utils.data import Dataset,DataLoader

In [2]:
number = torch.Tensor(list(range(10)))
number

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [4]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __getitem__(self, index):
        return self.data[index]
    def __len__(self):
        return len(self.data)

In [6]:
dataset = MyDataset(number)
dataloader = DataLoader(
                dataset,
                batch_size=2,
                shuffle=False,
                num_workers=0)


In [7]:
for idx, num in enumerate(dataloader):
    print(f"{idx},{num}")

0,tensor([0., 1.])
1,tensor([2., 3.])
2,tensor([4., 5.])
3,tensor([6., 7.])
4,tensor([8., 9.])


### 示例2：

In [8]:
import torch
from torch.utils.data import Dataset,DataLoader

In [14]:
imgs   = torch.rand(10,3,5,5)
labels = torch.rand(10,1)

In [54]:
class MyDataset(Dataset):
    def __init__(self, imgs,labels):
        self.imgs   = imgs
        self.labels = labels
        
    def __getitem__(self, index):
        img   = self.imgs[index]
        label = self.labels[index]
        #注意这种返回方式使得item=(img,label)是一个tuple类型
        return img,label
    
    def __len__(self):
        return self.imgs.shape[0]
    
    def collate_fn(self,batch):
        '''
        batch是一个list，每个成员是__getitem__反回的tuple=(img,label)，
        故需要对tuple进行处理，将img组合为新的tensor，将label组合为新的tensor
        '''
        #imgs, labels = list(zip(*batch))
        #imgs是包含tensor的tuple类型,   例如imgs=(img1,img2...)
        #labels是包含tensor的tuple类型，例如labels=(lbl1,lbl2...)
        imgs, labels = zip(*batch)
        #将tuple内的tensor叠在一起
        imgs   = torch.stack([img for img in imgs])
        labels = torch.stack([label for label in labels])
        '''
        对每个image第一行进行一些操作
        '''
        #imgs[:,:,0,:] = 2
        return imgs, labels

In [55]:
dataset = MyDataset(imgs,labels)
dataloader = DataLoader(
                dataset,
                batch_size=2,
                shuffle=False,
                num_workers=0,
                collate_fn=dataset.collate_fn)

In [57]:
for idx, (img,label) in enumerate(dataloader):
    print(f"<<idx>>{idx}")
    print(f"img shape={img.shape},label shape={label.shape}")
    

<<idx>>0
img shape=torch.Size([2, 3, 5, 5]),label shape=torch.Size([2, 1])
<<idx>>1
img shape=torch.Size([2, 3, 5, 5]),label shape=torch.Size([2, 1])
<<idx>>2
img shape=torch.Size([2, 3, 5, 5]),label shape=torch.Size([2, 1])
<<idx>>3
img shape=torch.Size([2, 3, 5, 5]),label shape=torch.Size([2, 1])
<<idx>>4
img shape=torch.Size([2, 3, 5, 5]),label shape=torch.Size([2, 1])
